<span style="color:red"><b><<<<<<< local</b></span>

In [9]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession, functions as F
import os
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

In [10]:
# Reading Parquet Data
sdf_3000 = spark.read.parquet('../data/raw/work_3000.parquet')

In [11]:
# now, we want to ensure everything has consistent casing to make our lives easier
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in sdf_3000.columns]
sdf_3000 = sdf_3000.select(*consistent_col_casing)

# this will be used in the cell below when reading in
sdf_schema = sdf_3000.schema
sdf_schema

StructType([StructField('url', StringType(), True), StructField('postcode', StringType(), True), StructField('suburb', StringType(), True), StructField('name', StringType(), True), StructField('cost_text', StringType(), True), StructField('beds', StringType(), True), StructField('baths', StringType(), True), StructField('parking', StringType(), True), StructField('property_type', StringType(), True)])

In [12]:
# Change data type
sdf = spark.read.schema(sdf_schema).parquet('../data/raw/*')

In [13]:
# Create new parquet of raw data
sdf \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet('../data/raw/raw_data.parquet')

24/09/17 13:36:27 WARN TaskSetManager: Stage 9 contains a task of very large size (3184 KiB). The maximum recommended task size is 1000 KiB.


In [31]:
# Path to your folder containing the files
folder_path = "../data/raw"

# List all files in the folder
all_files = os.listdir(folder_path)

# Iterate through each file
for file in all_files:
    # Construct full file path
    file_path = os.path.join(folder_path, file)
    
    # Check if it's a parquet file (adjust for other formats as needed)
    if file.endswith(".parquet"):
        # Read the file into a DataFrame
        df = spark.read.parquet(file_path)
        
        # Print the file name and its size (number of rows)
        print(f"File: {file}, Size (rows): {df.count()}")

    # For Parquet files
    elif file.endswith(".parquet"):
        df = spark.read.parquet(file_path)
        print(f"File: {file}, Size (rows): {df.count()}")
    
    # For CSV files (you can add more formats similarly)
    elif file.endswith(".csv"):
        df = spark.read.csv(file_path, header=True, inferSchema=True)
        print(f"File: {file}, Size (rows): {df.count()}")

File: work_3573.parquet, Size (rows): 222
File: work_3648.parquet, Size (rows): 122
File: work_3873.parquet, Size (rows): 35
File: work_3623.parquet, Size (rows): 17
File: work_3373.parquet, Size (rows): 385
File: work_3798.parquet, Size (rows): 14
File: work_3473.parquet, Size (rows): 56
File: work_3748.parquet, Size (rows): 43
File: work_3723.parquet, Size (rows): 30
File: work_3273.parquet, Size (rows): 38
File: work_3996.parquet, Size (rows): 583
File: work_3698.parquet, Size (rows): 73
File: work_3448.parquet, Size (rows): 106
File: work_3773.parquet, Size (rows): 236
File: work_3423.parquet, Size (rows): 31
File: work_3398.parquet, Size (rows): 22
File: work_3898.parquet, Size (rows): 54
File: property_metadata_chunk_1.parquet, Size (rows): 1444
File: work_3248.parquet, Size (rows): 137
File: work_3598.parquet, Size (rows): 14
File: work_3223.parquet, Size (rows): 1083
File: work_3548.parquet, Size (rows): 3
File: work_3823.parquet, Size (rows): 329
File: work_3673.parquet, Size 

24/09/17 03:40:46 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 304206 ms exceeds timeout 120000 ms
24/09/17 03:40:46 WARN SparkContext: Killing executors is not supported by current scheduler.
24/09/17 03:40:52 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

In [14]:
sdf = spark.read.parquet('../data/raw/raw_data.parquet')
print((sdf.count(), len(sdf.columns)))

(13363, 9)


In [29]:
sdf.sort('postcode')

url,postcode,suburb,name,cost_text,beds,baths,parking,property_type
https://www.domai...,3000,melbourne,4703/568 Collins ...,$750 a week and F...,2,2,1,Apartment / Unit ...
https://www.domai...,3000,melbourne,4307/639 Little L...,$600 and Fully Fu...,1,1,0,Apartment / Unit ...
https://www.domai...,3000,melbourne,1006/442 Elizabet...,$650 per week and...,1,1,0,Apartment / Unit ...
https://www.domai...,3000,melbourne,2213/27 Little Co...,$750 a week and F...,2,2,0,Apartment / Unit ...
https://www.domai...,3000,melbourne,99 Franklin Stree...,"Furnished, bills,...",1,1,0,Apartment / Unit ...
https://www.domai...,3000,melbourne,8301/228 La Trobe...,$750 **INSPECTION...,1,1,0,Apartment / Unit ...
https://www.domai...,3000,melbourne,1406/22 Jane Bell...,$795 include bill...,2,1,1,Apartment / Unit ...
https://www.domai...,3000,melbourne,103/19 Exploratio...,$540 Per Week Inc...,1,1,0,Studio
https://www.domai...,3000,melbourne,2514/23 Mackenzie...,$720 per week *Pa...,2,1,0,Apartment / Unit ...
https://www.domai...,3000,melbourne,1202/601 Little C...,"$620 per week, $2...",2,1,0,Apartment / Unit ...


<span style="color:red"><b>=======</b></span>

In [ ]:
def remove_first_postcode(chunk: int) -> None:
    """Function that removes the first postcode from affected chunks which have a double up
    
    Parameters:
    chunk - int type that mentions which chunk we will be changing

    Output:
    None - we are overwriting the parquet file wanted with the same contents without the first postcode
    """
    input_path = '../data/raw/work_{}.parquet'.format(chunk)
    
    df = spark.read.parquet(input_path)

    first_postcode = chunk - 25

    filtered_df = df.filter(df.postcode != first_postcode)

    filtered_df.write.mode("overwrite").parquet(input_path)


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("example").getOrCreate()

chunk_list = []
i = 3023
while i <= 3923:
    chunk_list.append(i)
    i += 25

for chunk in chunk_list:
    remove_first_postcode(chunk)

your 131072x1 screen size is bogus. expect trouble
24/09/18 00:56:29 WARN Utils: Your hostname, DESKTOP-RBVA59Q resolves to a loopback address: 127.0.1.1; using 172.26.88.196 instead (on interface eth0)
24/09/18 00:56:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/18 00:56:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[3023, 3048, 3073, 3098, 3123, 3148, 3173, 3198, 3223, 3248, 3273, 3298, 3323, 3348, 3373, 3398, 3423, 3448, 3473, 3498, 3523, 3548, 3573, 3598, 3623, 3648, 3673, 3698, 3723, 3748, 3773, 3798, 3823, 3848, 3873, 3898, 3923]


<span style="color:red"><b>>>>>>>> remote</b></span>